# Reconocimiento de comentarios negativos

En esta libreta vamos a investiar si, utilizando métodos sencillos y 
librerías existentes podemos detectar los comentarios negativos.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Para el tratamiento del lenguaje
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

# Para el aprendizaje (se agregarán algunas más conforme se necesite)
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
import sklearn.metrics as metricas


## Paso 1: Recuperar y tratar los datos para usarse

Los datos se encuentran en un directorio privado, los cuales ya se 
encuentran separados de un conjunto de prueba. 

Los datos se encuentran como una tabla tipo csv de la siguiente manera:

Columnas  |    Nombre       |     Descripción
----------|-----------------|----------------------------------------------------------
1         |   S/N           |    numeración (sin sentido)
2         |   id            |    Identificador del mensaje
3         |   create_time   |    Fecha de creación del mensaje
4         |   id_from       |    Identificador del usuario (emisor del mensaje)
5         |   comment       |    El mensaje en si mismo
6         |   like_count    |    Número de likes que recibió el mensaje
7         |   flag          |    Polaridad (0 -> Negativo, 1 -> Positivo, 2 -> Neutro)
8         |   ready         |    Revisado por un operador (1 -> No, 2 -> Si)

Una vez recuperados los datos, se realiza el siguiente pretratamiento:

1. La columna `flag` será el valor de salida y la columna `comment` 
   como variables de entrada, las cuales son string. 
   
2. La columna `comment`la vamos a tratar, ya que cada usuario maneja 
   codificaciones diferentes y los datos están hechos un chile con queso.

3. La columna `flag` se le asigna valor de 0 a los valores 1 y 2 
   (comentarios no negativos) y valor de 1 a los que tienen valor de 0
   (comentarios negativos). Convertimos el problema a un problema de
   clasificación binaria.
   
4. Los datos se separan, 80% para entrenamiento y 20% para validación, 
   procurando que en ambos conjuntos haya la misma proporción de 
   elementos de ambas clases.


In [3]:
# Cargamos los datos
df = pd.read_csv("privado/data_train.csv")

X = df["comment"]
X = X.apply(lambda x: str(x).decode('unicode_escape').encode('utf-8', 'ignore').strip())

y = np.array(df['flag'])

# Problema binario 1 = Comentario negativo
y = np.where(y > 0, 0, 1)

# Separamos 20% para validación
indices = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=0)
for tr_i, va_i in indices:
    X_train, X_valid = X.loc[tr_i], X.loc[va_i]
    y_train, y_valid = y[tr_i], y[va_i]
    break


## Paso 2: Genera la serie de operaciones a realizar.

En esta paso vamos a decidir cuales métodos vamos a utilizar para tratar la información,
y cual método vamos a utilizar para la clasificación. Los métodos que se seleccionen serán
incluídos en un `pipeline` que nos ayuda a serializar las operaciones a realizar.

Entre los métodos de vectorización de la información que podemos utilizar se encuentran:

1. Bolsa de palabras (o bigramas): `CountVectorizer`
2. Tf–idf term weighting: `TfidfVectorizer`

Entre los métodos de reducción de la dimensionalidad se encuentran:

1. Análisis en componentes principales para datos dispersos: `SparsePCA`
2. Latent semantic analysis (LSA): `TruncatedSVD`

Estos pueden ser utilizados en forma combinada. Los métodos de clasificación pueden ser:

1. Máquina de Vectores de Soporte Lineal: `LinearSVC`
2. Máquina de Vectores de Soporte: `SVC`
3. Naive Bayes: `MultinomialNB`
4. Bosques aleatorios: `RandomForestClassifier`
5. AdaBoost: `AdaBoostClassifier`
6. Árbol de desición: `DecisionTreeClassifier`

Por último, es necesario decidir si hay parámetros que pueden ser modificados, y en que términos pueden ser modificados, esto se agrega en el diccionario `parameters`.

In [4]:
espanol_stopwords = stopwords.words('spanish')
vectorizer = CountVectorizer(
    analyzer='word',
    lowercase=True,
    stop_words=espanol_stopwords)

# Pasos del procesamiento
pipeline = Pipeline([
     ('vect', vectorizer),
     ('cls', LinearSVC()),
    ])

# Aqui definimos el espacio de parámetros a explorar
parameters = {
    # 'vect__max_df': (0.9, 0.95, 1.0),
    # 'vect__min_df': (1, 0.05, 0.1),
    'vect__max_features': (20000, 40000),
    # 'vect__ngram_range': ((1, 1), (1, 2)),  # unigramas o bigramas
    'cls__C': (0.001, 0.01, 0.1),
    'cls__class_weight': ('balanced', None)
    # 'cls__loss': ('hinge', 'squared_hinge'),
    # 'cls__max_iter': (500, 1000)
}


## Paso 3. Ejecutar el script, buscando optimizar parámetros

Hacemos el paso completo, pero vamos a checar todas las combinaciones de parámetros 
para seleccionar la que reduzca un criterio de error. Entre los criterios de error
se encuentran `"recall"` y `"f1"` entre otros.


In [ ]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, scoring='recall')
grid_search.fit(X_train, y_train)

# Muestra los parametros seleccionados
print grid_search.best_params_


## Paso 4: Analisis de los resultados

Por último se observan los mejores resultados encontrados y analizamos que tan bien
se desempeña en los datos de validación.


In [ ]:
# Muestra los resultados con datos de validacion
y_est = grid_search.predict(X_valid)
print metricas.confusion_matrix(y_valid, y_est)
